In [32]:
import numpy as np
import math

In [35]:
NUCLEOTIDES = ['A', 'C', 'G', 'T']
STATES = ['E', '5', 'I']

# Initial Probabilities
INIT_PROB = {'E': 1.0, '5': 0.0, 'I': 0.0}

# Transition Probabilities
TRANS_PROB = {
    'Start': {'E':1.0, '5': 0, 'I': 0.0, 'End':0.0},
    'E': {'E': 0.9, '5': 0.1 , 'I': 0.0, 'End': 0.0},
    '5': {'E': 0.0, '5': 0.0 , 'I': 1.0, 'End': 0.0},
    'I': {'E': 0.0, '5': 0.0 , 'I': 0.9, 'End': 0.1}
}

# Emission Probabilities
EMISSION_PROB = {
    'E': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
    '5': {'A': 0.05, 'C': 0.00, 'G': 0.95, 'T': 0.00},
    'I': {'A': 0.4, 'C': 0.1, 'G': 0.1, 'T': 0.4},
}

def log(x):
    return -math.inf if x == 0 else math.log(x)

# Log Probability Calculation
def LOG_PROB(state_path, observed_sequence):
    if len(state_path) != len(observed_sequence):
        raise ValueError("The length of the observed and state sequences must match.")
    
    sequence_length = len(observed_sequence)
    log_probability = 0.0
    previous_state = 'Start'

    for index, (current_state, nucleotide) in enumerate(zip(state_path, observed_sequence)):
        transition_p = TRANS_PROB[previous_state][current_state]
        emission_p = EMISSION_PROB[current_state][nucleotide]
        log_probability += log(transition_p) + log(emission_p)
        previous_state = current_state

    # Final transition to 'End' state if last state is 'I'
    if previous_state == 'I':
        log_probability += log(TRANS_PROB[previous_state]['End'])

    return log_probability


In [36]:
def VITERBI(observed_sequence):
    num_states = len(STATES)
    sequence_length = len(observed_sequence)

    # Viterbi matrix to store log probabilities
    viterbi_matrix = np.full((num_states, sequence_length), -np.inf)

    # Backpointer matrix to track the optimal path
    backpointer = np.zeros((num_states, sequence_length), dtype=int)

    # Map state to index for easy reference
    state_index = {state: idx for idx, state in enumerate(STATES)}

    # Initialization: time step 0
    for i, state in enumerate(STATES):
        viterbi_matrix[i, 0] = log(INIT_PROB[state]) + log(EMISSION_PROB[state][observed_sequence[0]])

    # Dynamic programming: time steps 1 to T-1
    for t in range(1, sequence_length):
        for curr_idx, curr_state in enumerate(STATES):
            max_log_prob = -math.inf
            best_prev_idx = 0

            for prev_idx, prev_state in enumerate(STATES):
                transition_prob = log(TRANS_PROB[prev_state][curr_state])
                candidate_log_prob = viterbi_matrix[prev_idx, t - 1] + transition_prob

                if candidate_log_prob > max_log_prob:
                    max_log_prob = candidate_log_prob
                    best_prev_idx = prev_idx

            emission_prob = log(EMISSION_PROB[curr_state][observed_sequence[t]])
            viterbi_matrix[curr_idx, t] = max_log_prob + emission_prob
            backpointer[curr_idx, t] = best_prev_idx

    # Backtrack from the last column to build the best path
    best_last_state = np.argmax(viterbi_matrix[:, -1])
    best_path = [STATES[best_last_state]]

    for t in range(sequence_length - 1, 0, -1):
        best_last_state = backpointer[best_last_state, t]
        best_path.insert(0, STATES[best_last_state])

    # Return the best path and the highest log probability
    best_log_prob = np.max(viterbi_matrix[:, -1])
    return best_path, best_log_prob


In [38]:
# Testing on sequence
obs_seq = "CTTCATGTGAAAGCAGACGTAAGTCA"
most_likely_path, log_probability = VITERBI(obs_seq)

print("Most likely state path:")
print("".join(most_likely_path))
print("\nLog probability of this path:")
print(log_probability)


Most likely state path:
EEEEEEEEEEEEEEEEEEEEEEEEEE

Log probability of this path:
-38.677666280562796
